In [1]:
%run stdPackages.ipynb
read = {'variables': ['Fundamentals', 'Load', 'Generators_Other','TL','Regulation'], 
        'variable2D': ['Generators_FuelMix','HourlyVariation'],
        'scalars': ['Scalars'],
        'maps': ['Generators_Categories', 'Load_Categories']}
db = dbFromWB(os.path.join(d['data'],'E3.xlsx'), read)
readSets(db)

# The ```mBasicTrade.mRES``` model

### **The model**

The model is an extension to the [mBasicTrade](M_mBasicTrade.ipynb) model. The difference is that the share of electricity generation that is clean/green has a lower bound. We can enforce two types of: One common, or one for each geographic area.

**With one common cap:**

$$\begin{align}
    \sum_{id\in\text{cleanIds}}\sum_{h} E_{id, h} \geq& \overline{\text{RESCap}} \cdot \sum_{h,g} D_h^g \tag{1}
\end{align}$$
Written on the augmented linear programming form this becomes:
$$\begin{align}
    \overline{\text{RESCap}} \cdot \sum_{h,g}D_h^g -\sum_{id\in\text{cleanIds}}\sum_{h} E_{id, h} \leq 0. \tag{2}
\end{align}$$

Solve with common cap:

In [2]:
m = mBasicTrade.mRES(db,commonCap = True)
m.solve()

Solution status 0: Optimization terminated successfully.


**With geograpic-specific caps:**

In this case the following constraint is imposed for each area $g$:

$$\begin{align}
    \overline{RESCap}_g \cdot \sum_h D_h^g - \sum_{id\in \text{cleanIds} \text{ and }id\in\mathcal{I}_g} \sum_h E_{id,h} \leq 0.
\end{align}$$

Solve with geographic-specific caps:

In [3]:
m.commonCap = False
m.solve()

Solution status 0: Optimization terminated successfully.


### Adjustments to the code

Compared to ```mBasicTrade``` the main adjustment is that there is an added ```block``` of the type ```ub```:

The specification of the constraint is in terms of syntax equivalent to the ```eq``` constraints:
* There is a constraint named ```RESCapConstraint``` ,
* with a parmeter constraint 0,
* $A_{ub}$ coefficients for this constraint are:
    * $-1$ on the variable  ```'Generation'```, but only for the renewable ids (```self.cleanIds```), and
    * the parameter ```RESCap``` on the ```HourlyDemand``` variable.

**With one common cap:**

<img src="snippets/mBasicTrade_RES_snippet1.png" width="1200" height="400">

The parameter ```b``` is set to zero, ```Generation``` is added with coefficient $-1$ for ```self.cleanIds```, and ```HourlyDemand``` is added with the coefficient corresponding to the mean of entries in ```self.db['RESCap']``` (which is defined over $g$).

**With geographic-specific caps:**

<img src="snippets/mBasicTrade_RES_snippet2.png" width="1200" height="400">

In this case, the parameter constraint $b=0$ is repeated for all $g$:

In [4]:
pd.Series(0, index = m.db['RESCap'].index)

g
g1    0
g2    0
dtype: int64

The ```Generation``` variable is added with $-1$ (for ```self.cleanIds```). We add the index level ```g_alias``` as a copy of the level ```g``` to indicate that the coefficients are only used when the constraint index (```g_alias```) matches the variable index. The matrix $A$ becomes:

In [5]:
rc_pd(lpModels.appIndexWithCopySeries(pd.Series(-1, index = m.globalDomains['Generation']), 'g','g_alias'),
      m.cleanIds).unstack(level='g_alias').fillna(0)

g_alias    g1   g2
id  g  h          
id3 g1 1 -1.0  0.0
       2 -1.0  0.0
       3 -1.0  0.0
       4 -1.0  0.0
id4 g1 1 -1.0  0.0
       2 -1.0  0.0
       3 -1.0  0.0
       4 -1.0  0.0
id6 g2 1  0.0 -1.0
       2  0.0 -1.0
       3  0.0 -1.0
       4  0.0 -1.0

The ```HourlyDemand``` variable is added with coefficient ```RESCap```. The coefficient is broadcasted to fit the ```HourlyDemand``` variable:

In [6]:
s = lpCompiler.broadcast(m.db['RESCap'], m.globalDomains['HourlyDemand'])
s

g   h
g1  1    0.1
    2    0.1
    3    0.1
    4    0.1
g2  1    0.1
    2    0.1
    3    0.1
    4    0.1
dtype: object

Then, as with the ```Generation``` variable, we add the ```g_alias``` index to indicate what constraints the variable enters in: Only when ```g=g_alias```:

In [7]:
lpModels.appIndexWithCopySeries(s, 'g', 'g_alias')

g   h  g_alias
g1  1  g1         0.1
    2  g1         0.1
    3  g1         0.1
    4  g1         0.1
g2  1  g2         0.1
    2  g2         0.1
    3  g2         0.1
    4  g2         0.1
dtype: object